In [75]:
#import libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import json
import pandas as pd
from pandas.io.json import json_normalize
import os
import re
import random
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.neighbors import NearestNeighbors
#from nltk.stem.snowball import EnglishStemmer
#from sklearn.feature_extraction._stop_words import ENGLISH_STOP_WORDS

import nltk
import string
from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

In [76]:
#import file about the french museums infos

Paris_exhibitions=pd.read_csv('Cleaned_data/Paris_exhibitions.csv', sep=',', encoding='UTF8')

pd.set_option('display.max_columns', 20)

In [77]:
#gets user input

genre = input("Hi! What is the genre of the exhibition you're interested in?")
audience = input("Are you interested in a Public enfant / Public adolescent & adulte / Tout public / 'Public enfant & adolescent / Public adulte / Public adolescent exhibition?")

#uses user input to print out information
print("You're interested in", genre, "and by an exhibition for", audience)

You're interested in Art contemporain and by an exhibition for Tout public


In [78]:
exhibition_suggestion = Paris_exhibitions.loc[(Paris_exhibitions["genre"] == genre) & (Paris_exhibitions["audience"] == audience)]
exhibition_suggested = exhibition_suggestion.sample()
exhibition_suggested

,id,url,titre,chapeau,description,date_de_début,date_de_fin,mots_clés,nom_du_lieu,adresse_du_lieu,code_postal,ville,coordonnées_géographiques,type_de_prix,audience,genre,durée
908,35727,https://www.paris.fr/evenements/fredj-moussa-3...,Fredj Moussa - Solar Noon,"Dans un paysage désertique d'Afrique du Nord, ...","<p>Né en 1992 à Paris, Fredj Moussa travaille ...",2023-06-03 22:00:00+02:00,2023-06-04 05:00:00+02:00,"Art contemporain,Concert,vidéo,musique",Mairie du Centre,2 rue Eugène-Spuller,75003,Paris,"48.863992005989, 2.36153198516067",gratuit,Tout public,Art contemporain,0


In [79]:
#retrieve the index number of the exhibition suggested

index_suggested = exhibition_suggested.index.item()

#retrieve the exhibition information

exhibition_suggestion.loc[index_suggested:index_suggested]

,id,url,titre,chapeau,description,date_de_début,date_de_fin,mots_clés,nom_du_lieu,adresse_du_lieu,code_postal,ville,coordonnées_géographiques,type_de_prix,audience,genre,durée
908,35727,https://www.paris.fr/evenements/fredj-moussa-3...,Fredj Moussa - Solar Noon,"Dans un paysage désertique d'Afrique du Nord, ...","<p>Né en 1992 à Paris, Fredj Moussa travaille ...",2023-06-03 22:00:00+02:00,2023-06-04 05:00:00+02:00,"Art contemporain,Concert,vidéo,musique",Mairie du Centre,2 rue Eugène-Spuller,75003,Paris,"48.863992005989, 2.36153198516067",gratuit,Tout public,Art contemporain,0


In [80]:
#print out information about the exhibition suggested
print("Here is one exhibition you could like:\n", str(exhibition_suggested['titre']), "\nDescription:", str(exhibition_suggested['chapeau']), "\nPlace:", str(exhibition_suggested['nom_du_lieu']))

Here is one exhibition you could like:
 908    Fredj Moussa - Solar Noon
Name: titre, dtype: object 
Description: 908    Dans un paysage désertique d'Afrique du Nord, ...
Name: chapeau, dtype: object 
Place: 908    Mairie du Centre
Name: nom_du_lieu, dtype: object


# Machine learning

In [81]:
def find_similar_exhibitions(index_suggested):
    df = Paris_exhibitions.drop_duplicates(subset=["chapeau"]).reset_index(drop=True)
    strings = list(df["chapeau"])
    strings_without_digits = []

    # Uses a regex in order to remove the numbers
    pattern = r"\d+"

    for s in strings:
        if isinstance(s, str):
            s_without_digits = re.sub(pattern, "", s)
            strings_without_digits.append(s_without_digits)
        else:
            strings_without_digits.append(s)

    # Convert the keywords into a list
    keywords = list(df["mots_clés"])

    # Create CountVectorizer and fit_transform the texts
    vectorizer = CountVectorizer()
    texts = ["" if pd.isna(text) else text for text in strings_without_digits]
    vectors = vectorizer.fit_transform(texts)
    vocabulary = vectorizer.get_feature_names_out()

    # Create DataFrame with the vectors
    headers = pd.DataFrame.sparse.from_spmatrix(vectors, columns=vocabulary, index=texts)

    # Create CountVectorizer and fit_transform the keywords
    vectorizer = CountVectorizer()
    vectors = vectorizer.fit_transform(keywords)
    vocabulary = vectorizer.get_feature_names_out()

    # Create DataFrame with the vectors
    keywords = pd.DataFrame.sparse.from_spmatrix(vectors, columns=vocabulary, index=keywords)

    # Apply PCA on headers
    pca = PCA(n_components=15)  # Specify the number of components to keep
    vectors_pca = pca.fit_transform(headers.to_numpy())

    # Create DataFrame with PCA components for headers
    headers_pca = pd.DataFrame(vectors_pca, index=df.index)

    # Apply PCA on keywords
    pca = PCA(n_components=15)  # Specify the number of components to keep
    vectors_pca = pca.fit_transform(keywords.to_numpy())

    # Create DataFrame with PCA components for keywords
    keywords_pca = pd.DataFrame(vectors_pca, index=keywords.index)

    # Concatenate the PCA components
    concatenated_df = pd.concat([headers_pca, keywords_pca])

    # Fit Nearest Neighbors model
    neigh = NearestNeighbors(n_neighbors=6)
    neigh.fit(concatenated_df)

    # Find similar exhibitions for a specific index (35 in this case)
    distances, indices = neigh.kneighbors(concatenated_df.iloc[index_suggested:index_suggested+1])

    # Return the similar exhibitions
    return Paris_exhibitions.iloc[indices[0]]

In [82]:
find_similar_exhibitions(index_suggested)

,id,url,titre,chapeau,description,date_de_début,date_de_fin,mots_clés,nom_du_lieu,adresse_du_lieu,code_postal,ville,coordonnées_géographiques,type_de_prix,audience,genre,durée
908,35727,https://www.paris.fr/evenements/fredj-moussa-3...,Fredj Moussa - Solar Noon,"Dans un paysage désertique d'Afrique du Nord, ...","<p>Né en 1992 à Paris, Fredj Moussa travaille ...",2023-06-03 22:00:00+02:00,2023-06-04 05:00:00+02:00,"Art contemporain,Concert,vidéo,musique",Mairie du Centre,2 rue Eugène-Spuller,75003,Paris,"48.863992005989, 2.36153198516067",gratuit,Tout public,Art contemporain,0
965,34099,https://www.paris.fr/evenements/croqu-histoire...,Croqu'histoire : les vacances,Les bibliothécaires de la Bibliothèque Valeyre...,<p>Youpi l’heure des vacances va bientôt sonne...,2023-06-07 17:45:00+02:00,2023-06-07 18:45:00+02:00,"Enfants,Littérature",Centre Paris Anim' Paul Valeyre,24 rue Marguerite de Rochechouart,75009,Paris,"48.8779669855153, 2.34502198283795",gratuit,Public enfant,Enfants,0
1454,35850,https://www.paris.fr/evenements/conference-cha...,"Conférence ""Changé en pierre ! Une métamorphos...","""Changé en pierre ! Une métamorphose du troisi...",<p>À côté des métamorphoses animales et végéta...,2023-06-09 13:30:00+02:00,2023-06-09 15:30:00+02:00,Conférence,La passerelle,26 rue de Crussol,75011,Paris,"48.8645729976076, 2.3697900038072",payant,Public adulte,Conférence,0
54,34163,https://www.paris.fr/evenements/lectures-au-ja...,Lectures au jardin : La bibliothèque Diderot h...,"du 12 juillet au 23 août, mercredi matin de 10...","<p>Cet été, les bibliothécaires viennent à vot...",2023-07-12 13:30:00+02:00,2023-08-23 15:00:00+02:00,"Enfants,Littérature,Loisirs",kiosque du Square Trousseau,"2, rue Antoine Vollon",75012,Paris,"48.850193553992, 2.37670573519251",gratuit,Tout public,Enfants,42
1399,33807,https://www.paris.fr/evenements/cycle-cinema-d...,Cycle Cinéma du réel - projection de films doc...,"Cycle circulation du ""cinéma du réel"" - Projec...",<p>A la suite du 45e Festival international du...,2023-06-08 22:00:00+02:00,2023-06-08 23:30:00+02:00,Cinéma,Bibliothèque André Malraux,112 rue de Rennes,75006,Paris,"48.847914, 2.327471",gratuit,Tout public,Cinéma,0
1055,36542,https://www.paris.fr/evenements/a-vos-stylos-i...,Inscrivez-vous à la Grande Dictée des Champs-É...,"Dimanche 4 juin, la plus belle avenue du monde...","<p>À événement unique, décor unique.\nTableau ...",2023-06-04 02:00:00+02:00,2023-06-05 01:59:59+02:00,Littérature,Champs Élysée,Avenue des Champs Élysée,75007,Paris,"48.86906, 2.30993000000007",gratuit,Tout public,Littérature,0


# Is the exhibition suggested liked?

In [123]:
exhibition_suggestion

,id,url,titre,chapeau,description,date_de_début,date_de_fin,mots_clés,nom_du_lieu,adresse_du_lieu,code_postal,ville,coordonnées_géographiques,type_de_prix,audience,genre,durée
15,35206,https://www.paris.fr/evenements/exposition-ins...,Exposition Insectes,Exposition Insectes - Estampes et de dessins d...,<p>Exposition Insectes - Estampes et de dessin...,2023-04-14 17:00:00+02:00,2023-05-27 22:00:00+02:00,"Art contemporain,Expo,Peinture",Galerie Sagot - Le Garrec,10 rue de Buci,75006,Paris,"48.8538700197143, 2.33753996078741",gratuit,Tout public,Art contemporain,43
32,36425,https://www.paris.fr/evenements/bzh-photo-4-re...,BZH PHOTO · 4 regards sur le littoral breton à...,"Du 20 mai au 4 juin, BZH Photo réalise sa prem...","<p>Quatre résidents, quatre horizons, quatre s...",2023-05-20 15:00:00+02:00,2023-06-04 22:00:00+02:00,"Art contemporain,Expo,Photo",Galerie Esther Woerdehoff,36 rue Falguière,75015,Paris,"48.8423529913846, 2.3154200344884",gratuit,Tout public,Art contemporain,15
33,36527,https://www.paris.fr/evenements/ojan-the-live-...,"OJAN & The Live Drawing Project - CIELS, ARBRE...",Une marelle participative invite petits et gra...,<p>Architecte de formation et artiste montant ...,2023-06-03 22:00:00+02:00,2023-06-04 02:00:00+02:00,"Art contemporain,installation",Mairie du 18e arrondissement,1 place Jules Joffrin,75018,Paris,"48.8923039794398, 2.34468695816801",gratuit,Tout public,Art contemporain,0
37,35325,https://www.paris.fr/evenements/sylvain-gerver...,SYLVAIN GERVEREAU,COUP DE BAMBOU SUR LE CLIMAT / Installation vé...,"<p><strong>Féru\nd’histoire</strong>, surtout ...",2023-06-03 22:00:00+02:00,2023-06-04 04:00:00+02:00,"Art contemporain,Expo",Parc Raspail,21 rue Gallieni,94230,Cachan,"48.7954894507704, 2.3328964085598",gratuit,Tout public,Art contemporain,0
38,35568,https://www.paris.fr/evenements/parc-du-hangar...,Parc du Hangar Y - Déambulation artistique,Comprenant 27 œuvres de 20 artistes contempora...,"<p>Au Hangar Y, la programmation est conçue co...",2023-06-03 21:00:00+02:00,2023-06-03 03:00:00+02:00,"Art contemporain,Expo,Nature",Hangar Y,9 avenue de trivaux,92190,Meudon,"48.7988509977231, 2.23313899814764",gratuit,Tout public,Art contemporain,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1814,36742,https://www.paris.fr/evenements/parcours-nuit-...,Parcours Nuit Blanche à Poissy - Valerio Fasci...,Dans le cadre d'un parcours élaboré pour Nuit ...,"<p>Artiste autodidacte, Valerio Fasciani fonde...",2023-06-03 22:00:00+02:00,2023-06-04 01:00:00+02:00,"Art contemporain,photographie,installation",Maison de Fer,2ter Allée des Glaieuls,78300,Poissy,"48.9263120009221, 2.03218898830571",gratuit,Tout public,Art contemporain,0
1820,35303,https://www.paris.fr/evenements/bertrand-plane...,Bertrand Planes,IA Constellation - Le ciel étoilé vu sur la Se...,"<p><em>Ancien coder,\nartiste diplômé des Arts...",2023-06-04 01:00:00+02:00,2023-06-04 05:00:00+02:00,"Art contemporain,Musique",Pont de Saint-Ouen,"Les Berges, Pont de Saint-Ouen-sur-Seine",93400,Saint-Ouen-sur-Seine,"48.9185699931898, 2.32894000240086",gratuit,Tout public,Art contemporain,0
1825,35723,https://www.paris.fr/evenements/sandra-matamor...,"Sandra Matamoros, Back Home","Installation. Un cube miroir, lumineux et sono...","<p>Diplômée de l'ENSAD, Sandra Matamoros vit e...",2023-06-03 22:00:00+02:00,2023-06-03 03:00:00+02:00,"Art contemporain,installation",Église Saint-Ambroise,71 bis boulevard Voltaire,75011,Paris,"48.852139, 2.38956",gratuit,Tout public,Art contemporain,-1
1826,35769,https://www.paris.fr/evenements/la-taquinerie-...,La Taquinerie et Noir Vidéo - Marcel Reste Assis,"Une expérience immersive, sensorielle et décal...",<p>La Taquinerie est un collectif composé de 3...,2023-06-03 22:00:00+02:00,2023-06-04 05:00:00+02:00,"Art contemporain,Spectacle musical,installatio...",Bar à Bulles,4 Cité Véron,75018,Paris,"48.8845619926623, 2.33216003623426",gratuit,Tout public,Art contemporain,0


In [121]:
exhibition_suggested

,id,url,titre,chapeau,description,date_de_début,date_de_fin,mots_clés,nom_du_lieu,adresse_du_lieu,code_postal,ville,coordonnées_géographiques,type_de_prix,audience,genre,durée
908,35727,https://www.paris.fr/evenements/fredj-moussa-3...,Fredj Moussa - Solar Noon,"Dans un paysage désertique d'Afrique du Nord, ...","<p>Né en 1992 à Paris, Fredj Moussa travaille ...",2023-06-03 22:00:00+02:00,2023-06-04 05:00:00+02:00,"Art contemporain,Concert,vidéo,musique",Mairie du Centre,2 rue Eugène-Spuller,75003,Paris,"48.863992005989, 2.36153198516067",gratuit,Tout public,Art contemporain,0


In [134]:
new_exhibition_suggestion = exhibition_suggestion.drop(index=index_suggested)

new_exhibition_suggestion


,id,url,titre,chapeau,description,date_de_début,date_de_fin,mots_clés,nom_du_lieu,adresse_du_lieu,code_postal,ville,coordonnées_géographiques,type_de_prix,audience,genre,durée
15,35206,https://www.paris.fr/evenements/exposition-ins...,Exposition Insectes,Exposition Insectes - Estampes et de dessins d...,<p>Exposition Insectes - Estampes et de dessin...,2023-04-14 17:00:00+02:00,2023-05-27 22:00:00+02:00,"Art contemporain,Expo,Peinture",Galerie Sagot - Le Garrec,10 rue de Buci,75006,Paris,"48.8538700197143, 2.33753996078741",gratuit,Tout public,Art contemporain,43
32,36425,https://www.paris.fr/evenements/bzh-photo-4-re...,BZH PHOTO · 4 regards sur le littoral breton à...,"Du 20 mai au 4 juin, BZH Photo réalise sa prem...","<p>Quatre résidents, quatre horizons, quatre s...",2023-05-20 15:00:00+02:00,2023-06-04 22:00:00+02:00,"Art contemporain,Expo,Photo",Galerie Esther Woerdehoff,36 rue Falguière,75015,Paris,"48.8423529913846, 2.3154200344884",gratuit,Tout public,Art contemporain,15
33,36527,https://www.paris.fr/evenements/ojan-the-live-...,"OJAN & The Live Drawing Project - CIELS, ARBRE...",Une marelle participative invite petits et gra...,<p>Architecte de formation et artiste montant ...,2023-06-03 22:00:00+02:00,2023-06-04 02:00:00+02:00,"Art contemporain,installation",Mairie du 18e arrondissement,1 place Jules Joffrin,75018,Paris,"48.8923039794398, 2.34468695816801",gratuit,Tout public,Art contemporain,0
37,35325,https://www.paris.fr/evenements/sylvain-gerver...,SYLVAIN GERVEREAU,COUP DE BAMBOU SUR LE CLIMAT / Installation vé...,"<p><strong>Féru\nd’histoire</strong>, surtout ...",2023-06-03 22:00:00+02:00,2023-06-04 04:00:00+02:00,"Art contemporain,Expo",Parc Raspail,21 rue Gallieni,94230,Cachan,"48.7954894507704, 2.3328964085598",gratuit,Tout public,Art contemporain,0
38,35568,https://www.paris.fr/evenements/parc-du-hangar...,Parc du Hangar Y - Déambulation artistique,Comprenant 27 œuvres de 20 artistes contempora...,"<p>Au Hangar Y, la programmation est conçue co...",2023-06-03 21:00:00+02:00,2023-06-03 03:00:00+02:00,"Art contemporain,Expo,Nature",Hangar Y,9 avenue de trivaux,92190,Meudon,"48.7988509977231, 2.23313899814764",gratuit,Tout public,Art contemporain,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1814,36742,https://www.paris.fr/evenements/parcours-nuit-...,Parcours Nuit Blanche à Poissy - Valerio Fasci...,Dans le cadre d'un parcours élaboré pour Nuit ...,"<p>Artiste autodidacte, Valerio Fasciani fonde...",2023-06-03 22:00:00+02:00,2023-06-04 01:00:00+02:00,"Art contemporain,photographie,installation",Maison de Fer,2ter Allée des Glaieuls,78300,Poissy,"48.9263120009221, 2.03218898830571",gratuit,Tout public,Art contemporain,0
1820,35303,https://www.paris.fr/evenements/bertrand-plane...,Bertrand Planes,IA Constellation - Le ciel étoilé vu sur la Se...,"<p><em>Ancien coder,\nartiste diplômé des Arts...",2023-06-04 01:00:00+02:00,2023-06-04 05:00:00+02:00,"Art contemporain,Musique",Pont de Saint-Ouen,"Les Berges, Pont de Saint-Ouen-sur-Seine",93400,Saint-Ouen-sur-Seine,"48.9185699931898, 2.32894000240086",gratuit,Tout public,Art contemporain,0
1825,35723,https://www.paris.fr/evenements/sandra-matamor...,"Sandra Matamoros, Back Home","Installation. Un cube miroir, lumineux et sono...","<p>Diplômée de l'ENSAD, Sandra Matamoros vit e...",2023-06-03 22:00:00+02:00,2023-06-03 03:00:00+02:00,"Art contemporain,installation",Église Saint-Ambroise,71 bis boulevard Voltaire,75011,Paris,"48.852139, 2.38956",gratuit,Tout public,Art contemporain,-1
1826,35769,https://www.paris.fr/evenements/la-taquinerie-...,La Taquinerie et Noir Vidéo - Marcel Reste Assis,"Une expérience immersive, sensorielle et décal...",<p>La Taquinerie est un collectif composé de 3...,2023-06-03 22:00:00+02:00,2023-06-04 05:00:00+02:00,"Art contemporain,Spectacle musical,installatio...",Bar à Bulles,4 Cité Véron,75018,Paris,"48.8845619926623, 2.33216003623426",gratuit,Tout public,Art contemporain,0


In [182]:
def recommend_exhibition(exhibition_suggested):
    index = 0  # Counter to keep track of the suggested exhibitions
    while True:
        if index < len(exhibition_suggested):
            new_exhibition_suggestion = exhibition_suggested.iloc[index]
            while True:
                response = input("Is the suggested exhibition relevant for you? Yes/No (Y/N): ")
                if response.lower() == "y":
                    similar_exhibitions = find_similar_exhibitions(index_suggested)
                    return pd.DataFrame(similar_exhibitions)
                elif response.lower() == "n":
                    new_exhibition_suggestion = exhibition_suggestion.drop(index=index_suggested)
                    new_exhibition_suggested = new_exhibition_suggestion.sample()
                    print("Here is another exhibition you could like:\n")
                    print("Title:", new_exhibition_suggested['titre'].values[0])
                    print("Description:", new_exhibition_suggested['chapeau'].values[0])
                    print("Location:", new_exhibition_suggested['nom_du_lieu'].values[0])
                    response = input("Is the suggested exhibition relevant for you? Yes/No (Y/N): ")
                break
            else:
                print("Invalid response. Please enter 'Y' for Yes or 'N' for No.")
    else:
        print("No more exhibitions to suggest.")
        return


# Call the function and pass the exhibition DataFrame
recommend_exhibition(exhibition_suggested)


Here is another exhibition you could like:

Title: MANYAK
Description: DÉTOURNEMENTS / Installation / Création monumentale d’art urbain - Graf en perspectives
Location: Parc Raspail


,id,url,titre,chapeau,description,date_de_début,date_de_fin,mots_clés,nom_du_lieu,adresse_du_lieu,code_postal,ville,coordonnées_géographiques,type_de_prix,audience,genre,durée
908,35727,https://www.paris.fr/evenements/fredj-moussa-3...,Fredj Moussa - Solar Noon,"Dans un paysage désertique d'Afrique du Nord, ...","<p>Né en 1992 à Paris, Fredj Moussa travaille ...",2023-06-03 22:00:00+02:00,2023-06-04 05:00:00+02:00,"Art contemporain,Concert,vidéo,musique",Mairie du Centre,2 rue Eugène-Spuller,75003,Paris,"48.863992005989, 2.36153198516067",gratuit,Tout public,Art contemporain,0
965,34099,https://www.paris.fr/evenements/croqu-histoire...,Croqu'histoire : les vacances,Les bibliothécaires de la Bibliothèque Valeyre...,<p>Youpi l’heure des vacances va bientôt sonne...,2023-06-07 17:45:00+02:00,2023-06-07 18:45:00+02:00,"Enfants,Littérature",Centre Paris Anim' Paul Valeyre,24 rue Marguerite de Rochechouart,75009,Paris,"48.8779669855153, 2.34502198283795",gratuit,Public enfant,Enfants,0
1454,35850,https://www.paris.fr/evenements/conference-cha...,"Conférence ""Changé en pierre ! Une métamorphos...","""Changé en pierre ! Une métamorphose du troisi...",<p>À côté des métamorphoses animales et végéta...,2023-06-09 13:30:00+02:00,2023-06-09 15:30:00+02:00,Conférence,La passerelle,26 rue de Crussol,75011,Paris,"48.8645729976076, 2.3697900038072",payant,Public adulte,Conférence,0
54,34163,https://www.paris.fr/evenements/lectures-au-ja...,Lectures au jardin : La bibliothèque Diderot h...,"du 12 juillet au 23 août, mercredi matin de 10...","<p>Cet été, les bibliothécaires viennent à vot...",2023-07-12 13:30:00+02:00,2023-08-23 15:00:00+02:00,"Enfants,Littérature,Loisirs",kiosque du Square Trousseau,"2, rue Antoine Vollon",75012,Paris,"48.850193553992, 2.37670573519251",gratuit,Tout public,Enfants,42
1399,33807,https://www.paris.fr/evenements/cycle-cinema-d...,Cycle Cinéma du réel - projection de films doc...,"Cycle circulation du ""cinéma du réel"" - Projec...",<p>A la suite du 45e Festival international du...,2023-06-08 22:00:00+02:00,2023-06-08 23:30:00+02:00,Cinéma,Bibliothèque André Malraux,112 rue de Rennes,75006,Paris,"48.847914, 2.327471",gratuit,Tout public,Cinéma,0
1055,36542,https://www.paris.fr/evenements/a-vos-stylos-i...,Inscrivez-vous à la Grande Dictée des Champs-É...,"Dimanche 4 juin, la plus belle avenue du monde...","<p>À événement unique, décor unique.\nTableau ...",2023-06-04 02:00:00+02:00,2023-06-05 01:59:59+02:00,Littérature,Champs Élysée,Avenue des Champs Élysée,75007,Paris,"48.86906, 2.30993000000007",gratuit,Tout public,Littérature,0


In [184]:
def recommend_exhibition(exhibition_suggested):
    index = 0  # Counter to keep track of the suggested exhibitions
    while True:
        if index < len(exhibition_suggested):
            new_exhibition_suggestion = exhibition_suggested.iloc[index]
            while True:
                response = input("Is the suggested exhibition relevant for you? Yes/No (Y/N): ")
                if response.lower() == "y":
                    similar_exhibitions = find_similar_exhibitions(index_suggested)
                    return pd.DataFrame(similar_exhibitions)
                elif response.lower() == "n":
                    new_exhibition_suggestion = exhibition_suggestion.drop(index=index_suggested)
                    new_exhibition_suggested = new_exhibition_suggestion.sample()
                    print("Here is another exhibition you could like:\n")
                    print("Title:", new_exhibition_suggested['titre'].values[0])
                    print("Description:", new_exhibition_suggested['chapeau'].values[0])
                    print("Location:", new_exhibition_suggested['nom_du_lieu'].values[0])
                    response = input("Is the suggested exhibition relevant for you? Yes/No (Y/N): ")
                    if response.lower() == "y":
                        similar_exhibitions = find_similar_exhibitions(index_suggested)
                        return pd.DataFrame(similar_exhibitions)
                    elif response.lower() == "n":
                        new_exhibition_suggestion = exhibition_suggestion.drop(index=index_suggested)
                        new_exhibition_suggested = new_exhibition_suggestion.sample()
                        print("Here is another exhibition you could like:\n")
                        print("Title:", new_exhibition_suggested['titre'].values[0])
                        print("Description:", new_exhibition_suggested['chapeau'].values[0])
                        print("Location:", new_exhibition_suggested['nom_du_lieu'].values[0])
                    break
                else:
                    print("Invalid response. Please enter 'Y' for Yes or 'N' for No.")
        else:
            print("No more exhibitions to suggest.")
            return


# Call the function and pass the exhibition DataFrame
recommend_exhibition(exhibition_suggested)


Here is another exhibition you could like:

Title: Shana Rouaix - Si c'est un jour, allons-nous-en
Description: Des gouttes d'eau tombent sur un tambourin. La verticalité accueille le vide et leurs sons.
Location: Chapelle Saint-Louis de la Salpêtrière
Here is another exhibition you could like:

Title: Emmanuelle Léonard, Le Déploiement
Description: Rendez-vous dans le Grand nord canadien avec "Le Déploiement", l'exposition de l'artiste Emmanuelle Léonard au Centre culturel canadien à Paris, du 9 juin au 14 novembre.
Location: Centre culturel canadien


,id,url,titre,chapeau,description,date_de_début,date_de_fin,mots_clés,nom_du_lieu,adresse_du_lieu,code_postal,ville,coordonnées_géographiques,type_de_prix,audience,genre,durée
908,35727,https://www.paris.fr/evenements/fredj-moussa-3...,Fredj Moussa - Solar Noon,"Dans un paysage désertique d'Afrique du Nord, ...","<p>Né en 1992 à Paris, Fredj Moussa travaille ...",2023-06-03 22:00:00+02:00,2023-06-04 05:00:00+02:00,"Art contemporain,Concert,vidéo,musique",Mairie du Centre,2 rue Eugène-Spuller,75003,Paris,"48.863992005989, 2.36153198516067",gratuit,Tout public,Art contemporain,0
965,34099,https://www.paris.fr/evenements/croqu-histoire...,Croqu'histoire : les vacances,Les bibliothécaires de la Bibliothèque Valeyre...,<p>Youpi l’heure des vacances va bientôt sonne...,2023-06-07 17:45:00+02:00,2023-06-07 18:45:00+02:00,"Enfants,Littérature",Centre Paris Anim' Paul Valeyre,24 rue Marguerite de Rochechouart,75009,Paris,"48.8779669855153, 2.34502198283795",gratuit,Public enfant,Enfants,0
1454,35850,https://www.paris.fr/evenements/conference-cha...,"Conférence ""Changé en pierre ! Une métamorphos...","""Changé en pierre ! Une métamorphose du troisi...",<p>À côté des métamorphoses animales et végéta...,2023-06-09 13:30:00+02:00,2023-06-09 15:30:00+02:00,Conférence,La passerelle,26 rue de Crussol,75011,Paris,"48.8645729976076, 2.3697900038072",payant,Public adulte,Conférence,0
54,34163,https://www.paris.fr/evenements/lectures-au-ja...,Lectures au jardin : La bibliothèque Diderot h...,"du 12 juillet au 23 août, mercredi matin de 10...","<p>Cet été, les bibliothécaires viennent à vot...",2023-07-12 13:30:00+02:00,2023-08-23 15:00:00+02:00,"Enfants,Littérature,Loisirs",kiosque du Square Trousseau,"2, rue Antoine Vollon",75012,Paris,"48.850193553992, 2.37670573519251",gratuit,Tout public,Enfants,42
1399,33807,https://www.paris.fr/evenements/cycle-cinema-d...,Cycle Cinéma du réel - projection de films doc...,"Cycle circulation du ""cinéma du réel"" - Projec...",<p>A la suite du 45e Festival international du...,2023-06-08 22:00:00+02:00,2023-06-08 23:30:00+02:00,Cinéma,Bibliothèque André Malraux,112 rue de Rennes,75006,Paris,"48.847914, 2.327471",gratuit,Tout public,Cinéma,0
1055,36542,https://www.paris.fr/evenements/a-vos-stylos-i...,Inscrivez-vous à la Grande Dictée des Champs-É...,"Dimanche 4 juin, la plus belle avenue du monde...","<p>À événement unique, décor unique.\nTableau ...",2023-06-04 02:00:00+02:00,2023-06-05 01:59:59+02:00,Littérature,Champs Élysée,Avenue des Champs Élysée,75007,Paris,"48.86906, 2.30993000000007",gratuit,Tout public,Littérature,0
